In [5]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from encoder import encode_user_profile

In [6]:
def compare_profiles(user1_answers, user2_answers):
    # Encode user responses
    qc = QuantumCircuit(6, 3) # 6 qubits and 3 classical bits for measurement
    
    # Encode answers
    user1_circuit = encode_user_profile(user1_answers)
    user2_circuit = encode_user_profile(user2_answers)

    # Add these circuits to the main circuit
    qc.compose(user1_circuit, [0,1,2], inplace=True)
    qc.compose(user2_circuit, [3,4,5], inplace=True)

    # Comparison logic
    for i in range(3):
        qc.cx(i, i+3)   # CNOT with user2's qubits
        qc.x(i+3)       # X gate to flip the ancilla qubit

    # Measurement
    qc.measure([3,4,5],[0,1,2])

    return qc

In [7]:
user_profiles = [
    [1, 0, 1],  # User 1
    [0, 1, 0],  # User 2
    [1, 1, 0],  # User 3
    [0, 0, 1],  # User 4
    [1, 0, 0]   # User 5
]

In [9]:
from itertools import combinations

# prepare the simulator
aer_sim = Aer.get_backend('aer_simulator')

pairwise_results = {}

for idx, (user1, user2) in enumerate(combinations(range(len(user_profiles)), 2)):
    comparison_circuit = compare_profiles(user_profiles[user1], user_profiles[user2])
    transpiled_comparison_circuit = transpile(comparison_circuit, aer_sim)
    result = aer_sim.run(transpiled_comparison_circuit).result()
    counts = result.get_counts()
    pairwise_results[(user1, user2)] = counts

print(pairwise_results)

{(0, 1): {'000': 1024}, (0, 2): {'001': 1024}, (0, 3): {'110': 1024}, (0, 4): {'011': 1024}, (1, 2): {'110': 1024}, (1, 3): {'001': 1024}, (1, 4): {'100': 1024}, (2, 3): {'000': 1024}, (2, 4): {'101': 1024}, (3, 4): {'010': 1024}}


In [12]:
# classical post-processing

best_matches = []

match_threshold = 2 # at least 2 out of 3 answers match

for (user1, user2), counts in pairwise_results.items():
    for match_res in counts:
        match_count = match_res.count('1')
        if match_count >= match_threshold:
            best_matches.append((user1 + 1, user2 + 1))  # Adding 1 to adjust for 0-based indexing

print(best_matches)

[(1, 4), (1, 5), (2, 3), (3, 5)]
